In [1]:
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#pd.options.display.max_rows = 999
#pd.get_option("display.max_rows")

In [3]:
# Control whether to import the original data. 
import_original_data = True

Mila:

OK only four files needed to be corrected. I put them in your Meta4 folder under the name Elviz_Corrected

The remaining four files indeed contained fewer contigs.

Thanks!


In [4]:
import sys
print sys.executable 
print (sys.version)

/usr/bin/python
2.7.5 (default, Mar  9 2014, 22:15:05) 
[GCC 4.2.1 Compatible Apple LLVM 5.0 (clang-500.0.68)]


In [5]:
sns.set(style="whitegrid")

In [6]:
# Import the csv that translates the 127_HOW14 type labels to weeks and replicates.

from elviz_utils import IMPORT_METAINFO_TYPES, read_sample_info
sample_info = read_sample_info()

sample_info.head()

,ID,oxy,rep,week,project
0,1_LOW4,Low,1,4,1056013
1,13_LOW5,Low,1,5,1056037
2,25_LOW6,Low,1,6,1056061
3,37_LOW7,Low,1,7,1056085
4,49_LOW8,Low,1,8,1056109


In [7]:
# loop over each file, collapse rows that share phylogeny data to the genus label,
# save new .csv files, then append them all together in a separate step. 
#filepath = '/Volumes/Shares/Users/Janet/meta4/160121_elviz_files_updated/'
filepath = './data'
elviz_files = [f for f in os.listdir(filepath) if ".csv" in f]
print len(elviz_files)

88


In [8]:
from abundance_utils import read_and_reduce_elviz_csv

In [9]:
# test read_and_reduce_elviz_csv
tmp = read_and_reduce_elviz_csv(filename = elviz_files[0], 
                                filepath=filepath,
                               sample_info=sample_info)

In [10]:
tmp.head()

,Kingdom,Phylum,Class,Order,Family,Genus,abundance,ID,project,oxy,rep,week
642,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,0.275835,1_LOW4,1056013,NaN,NaN,NaN
494,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,0.266572,1_LOW4,1056013,NaN,NaN,NaN
198,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,0.066042,1_LOW4,1056013,NaN,NaN,NaN
650,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,other,0.057879,1_LOW4,1056013,NaN,NaN,NaN
478,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,other,0.036590,1_LOW4,1056013,NaN,NaN,NaN


In [11]:
# import the raw or raw-ish data. 
if import_original_data:
    filepath = '/Users/janet/Documents/Lidstrom Lab Work/meta4/160125_Elviz_files_corrected'
    elviz_files = [f for f in os.listdir(filepath) if ".csv" in f]
    print elviz_files[1:4]

['elviz_1056016_2_LOW4.csv', 'elviz_1056019_3_LOW4.csv', 'elviz_1056022_4_LOW4.csv']


In [12]:
if not os.path.exists('./results'):
    os.makedirs('./results/')
if not os.path.exists('./plots'):
    os.makedirs('./plots/')

In [13]:
from abundance_utils import read_and_reduce_all

In [14]:
if import_original_data:
    data_reduced = read_and_reduce_all(filename_list=elviz_files,
                                       filepath=filepath,
                                       sample_info = sample_info)
    print data_reduced.size  
else: 
    data_reduced = pd.read_csv("./results/reduced_data--all_phylogeny_remains.csv")

data_reduced.head()

NameError: global name 'filepath' is not defined

In [ ]:
sample_info_augmented.sort_values(by='project',)
sample_info_augmented.to_csv('data/sample_meta_info.tsv', sep='\t', index=False)

In [ ]:
# Should have only 1 row per Id now. 
data_reduced[data_reduced['Genus']=='Methylophilus']

if import_original_data:
    data_reduced.to_csv("./results/reduced_data--all_phylogeny_remains.csv", index=False)

In [ ]:
data_reduced.columns

In [ ]:
# make sure the abundances all sum to 1. 
for t, d in data_reduced.groupby('ID'):
    print t, ":", d['abundance'].sum()

In [ ]:
data_reduced.tail()

In [ ]:
# check number of sample imported.  Dave had a problem with the raw files 1/16/2015 using Python 3. 
len(data_reduced.ID.unique())

In [ ]:
data_reduced['sample_name'] = 'replicate '+ data_reduced['rep'].astype(str) +": " + data_reduced['oxy'] + ' O2'
print data_reduced['sample_name'].unique()

In [ ]:
data_reduced.tail()

In [ ]:
# Write an excel for Mila.  One notebook for replicate.  Each page is a different ID/week. 
by_repl_and_week = data_reduced.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')

In [ ]:
# The groupby returns tuples of the conditions and the data in a tuple. 
#for (rep, week, oxy), d in by_repl_and_week:
#    print "rep_" + str(rep)
#    print "week_" + str(week)
#    print oxy
#    print ""

In [ ]:
data_reduced['rep'].unique()

In [ ]:
# write same dictionary in a loop
excel_files = {}
for ox in data_reduced['oxy'].unique():
    for re in data_reduced['rep'].unique():
        print ox, re
        excel_files[(ox, re)] =  'elviz_binned--{}O2_rep{}.xlsx'.format(ox, re)
  
print excel_files

In [ ]:
# Turn off re-writ here:

write_excel = True
if import_original_data and write_excel:
#if write_excel:
    # make an excel writer for each file. 
    writer_dict = {}
    for oxy in data_reduced['oxy'].unique():
        for rep in data_reduced['rep'].unique():
            #print ox, re
            filename = 'elviz--{}O2_rep_{}.xlsx'.format(oxy, rep)
            #print filename
            writer_dict[(oxy, rep)] = pd.ExcelWriter(filename, engine='xlsxwriter')
            #print writer_dict
    print writer_dict
    
    # The groupby returns tuples of the conditions. 
    by_repl_and_week = data_reduced.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')  
    for (rep, week, oxy), d in by_repl_and_week:
        # use the writer that matches the replicate:
        print rep, week, oxy
        writer = writer_dict[(oxy, rep)]
        sheet_name = oxy + '_O2' +"_rep_" + str(rep)+"_week_" + str(week)
        print sheet_name
        d.reset_index()
        #print d.columns
        del d['index']
        #print d.columns
        d.to_excel(writer, sheet_name=sheet_name, index=False)
        #writer.save()
        print ""
    
    # close each writer.  This saves them. 
    for w_dict in writer_dict.values():
        w_dict.close()


In [ ]:
data_reduced.week.head()

In [ ]:
data_reduced.head()

In [ ]:
# Another check that we have each ID's abundances summing to 1. 
for t, d in data_reduced.groupby('ID'):
    if d['abundance'].sum() > 1.0001:
        print t, ":", d['abundance'].sum()

In [ ]:
data_reduced.head()

In [ ]:
# Further reduce data so only Genus is preserved. 
# drop the reads per kilobase or we will get more than 1 row for other.
data_reduced_genus = data_reduced.copy()
#del data_reduced_genus['reads per kilobase']
print data_reduced_genus.head()
data_reduced_genus = data_reduced_genus.groupby(['ID','rep','week','oxy','Genus']).sum().reset_index()
data_reduced_genus.sort_values(by=['rep', 'abundance'], inplace=True, ascending=False)
data_reduced_genus.tail(8)

In [ ]:
data_reduced_genus.columns

In [ ]:
data_reduced_genus.head()

In [ ]:
print data_reduced_genus[['rep','week', 'ID']].drop_duplicates()

In [ ]:
data_reduced_genus.head(3)

In [ ]:
by_repl_and_week_Genus = data_reduced_genus.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')
# check that replicates and week are in the right order: 
#for tup, d in by_repl_and_week_Genus:
#    print tup

In [ ]:
# Turn off re-writ here:
write_excel = True

if write_excel:
    # make an excel writer for each file. 
    writer_dict = {}
    for oxy in data_reduced['oxy'].unique():
        for rep in data_reduced['rep'].unique():
            #print ox, re
            filename = 'elviz--Genus_only--{}O2_rep_{}.xlsx'.format(oxy, rep)
            #print filename
            writer_dict[(oxy, rep)] = pd.ExcelWriter(filename, engine='xlsxwriter')
            #print writer_dict
    print writer_dict
    
    # The groupby returns tuples of the conditions. 
    by_repl_and_week = data_reduced_genus.groupby(['rep','week','oxy'])#.to_excel(writer,sheet_name='Sheet1')  
    for (rep, week, oxy), d in by_repl_and_week:
        # use the writer that matches the replicate:
        print rep, week, oxy
        writer = writer_dict[(oxy, rep)]
        sheet_name = oxy + '_O2' +"_rep_" + str(rep)+"_week_" + str(week)
        print sheet_name
        d.reset_index(inplace=True)
        del d['index']
        #print d.columns
        del d['level_0']
        d.to_excel(writer, sheet_name=sheet_name, index=False)
        #writer.save()
        print ""
    
    # close each writer.  This saves them. 
    for w_dict in writer_dict.values():
        w_dict.close()


In [ ]:
# Demo of simple plot.  Trim rows with abundance some percent.  Plot only one week/oxygen combo. 
def plot_by_abundance(data, abundance):
    d_plot = data[data['abundance'] > abundance]
    d_plot = pd.pivot_table(d_plot, values='abundance', index=['week'], columns='Genus') #.reset_index()
    print d_plot.head()
    d_plot.plot(kind='bar',stacked=True)
    sns.despine()
    #plt.legend(loc=2)
plot_by_abundance(data=data_reduced_genus[
        np.logical_and(
            data_reduced_genus['rep']==1, 
            data_reduced_genus['oxy']=="Low")], abundance=0.02)

In [ ]:
# Begin to generalize.  Want to be able to subset by oxygen and replicate. 
def subset_data_by_rep_and_oxy(data, rep, oxy):
    return data[
        np.logical_and(
            data_reduced_genus['rep']==rep, 
            data_reduced_genus['oxy']==oxy)]

In [ ]:
# Make an individual plot with the new subset_data_by_rep_and_oxy() function
def plot_by_abundance(data, abundance):
    d_plot = data[data['abundance'] > abundance]
    d_plot = pd.pivot_table(d_plot, values='abundance', index=['week'], columns='Genus') #.reset_index()
    #print d_plot.head()
    d_plot.plot(kind='bar',stacked=True)
    sns.despine()
    #plt.legend(loc=2)
plot_by_abundance(subset_data_by_rep_and_oxy(data= data_reduced_genus, rep=1, oxy='High'), abundance=0.02)

In [ ]:
# set up a figure with four subplots. 
fig, axs = plt.subplots(2, 4, figsize=(8, 6))  # ax belongs to fig. 
print fig
print axs

In [ ]:
g = data_reduced_genus.groupby(['rep', 'oxy'])
for t, dg in g:
    print t #, dg
    #print t[0]

In [ ]:
row_dict = {(1, 'High') : 2,
                (1, 'Low')  : 1,
                (2, 'High') : 2,
                (2, 'Low')  : 1,
                (3, 'High') : 2,
                (3, 'Low')  : 1,
                (4, 'High') : 2,
                (4, 'Low')  : 1}

In [ ]:
print axs
print ""
axs[(1-1)][2-1]

In [ ]:
#axd = {1:axs[0], 2:axs[1], 3:axs[2], 4:axs[3]}
title_list = ''

for t, df in g:
    #print t
    #print 'plot row:', row_dict[t]
    #print 'plot column:', t[0]
    df.plot(kind='scatter', ax=axs[row_dict[t]-1][t[0]-1], x='week', y='abundance', title='meow')

sns.despine()  # Removes the boxes around the plots.

In [ ]:
fig

In [ ]:
# set up a figure with four subplots. 
fig, axs = plt.subplots(2, 4, figsize=(12, 10))  # ax belongs to fig. 
#print fig
#print axs

# set up groupby 

d_plot = data_reduced_genus[data_reduced_genus['abundance'] > 0.05]
g = d_plot.groupby(['rep', 'oxy'])

#axd = {1:axs[0], 2:axs[1], 3:axs[2], 4:axs[3]}
title_list = ''

row_dict = {(1, 'High') : 2, (2, 'High') : 2, (3, 'High') : 2, (4, 'High') : 2,
            (1, 'Low')  : 1, (2, 'Low')  : 1, (3, 'Low')  : 1, (4, 'Low')  : 1}

for tup, df in g:
    # cast the data
    #print tup # tuple
    data_pivoted = pd.pivot_table(df, values='abundance', 
                                        index=['week'],  # drops all the other columns. 
                                        columns='Genus') 
    #print data_pivoted.head()
    #df.groupby('Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'ID', 'oxy', 'rep')
    #print axs[row_dict[tup]-1][tup[0]-1]
    data_pivoted.plot(kind='bar', ax=axs[row_dict[tup]-1][tup[0]-1], stacked=True)  # don't specify x b/c there is only 1 index. 


In [ ]:
# fig  # shows the figure.

In [ ]:
## We can get the last one out, but we want all of them out and want them to have shared colors. 
# set up a figure with four subplots. 
fig, axs = plt.subplots(2, 4, figsize=(12, 10))  # ax belongs to fig. 
#print fig
print axs

# set up groupby 

d_plot = data_reduced_genus[data_reduced_genus['abundance'] > 0.05]
g = d_plot.groupby(['rep', 'oxy'])

#axd = {1:axs[0], 2:axs[1], 3:axs[2], 4:axs[3]}
title_list = ''

row_dict = {(1, 'High') : 2, (2, 'High') : 2, (3, 'High') : 2, (4, 'High') : 2,
            (1, 'Low')  : 1, (2, 'Low')  : 1, (3, 'Low')  : 1, (4, 'Low')  : 1}

for tup, df in g:
    # cast the data
    #print tup # tuple
    data_pivoted = pd.pivot_table(df, values='abundance', 
                                        index=['week'],  # drops all the other columns. 
                                        columns='Genus') 
    #print data_pivoted.head()
    #df.groupby('Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'ID', 'oxy', 'rep')
    #print axs[row_dict[tup]-1][tup[0]-1]
    data_pivoted.plot(kind='bar', ax=axs[row_dict[tup]-1][tup[0]-1], stacked=True)  # don't specify x b/c there is only 1 index. 

plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.1,1,1),
            bbox_transform = plt.gcf().transFigure )            
#legend = plt.legend(loc='upper center', shadow=True, fontsize='x-large')   
#fig.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0.)
#         # it will place the legend on the outer right-hand side of the last axes    

In [ ]:
import itertools
methods=['method 1', 'method2', 'method 3', 'method 4']
times = range(0, 100, 10)
data = pd.DataFrame(list(itertools.product(methods, times, times)))
data.columns = ['method', 'dtsi','rtsi']
data['nw_score'] = np.random.sample(data.shape[0])
print data.head()

In [ ]:
# preview of pivot that happens:  (note use of pivot_table, not pivot.  Aggregating across duplicate rows somehow.)
data.pivot_table(index="dtsi", columns='rtsi', values='nw_score').head()

In [ ]:
data['dtsi'].unique()  # x-values
# rtsi are y-values
# cast 

In [ ]:
data_reduced_genus.head()

In [ ]:
# Make a heat map.  Then we aren't confused by colors, and overwhelmed by the # of labels. 
d_plot = data_reduced_genus[data_reduced_genus['abundance'] > 0.05]

def facet_heatmap(data, color, **kws):
    data = data.pivot(index='Genus', columns='week', values='abundance')
    sns.heatmap(data, cmap='Blues', **kws)  # <-- Pass kwargs to heatmap

with sns.plotting_context(font_scale=5.5):
    g = sns.FacetGrid(d_plot[d_plot['oxy']=='Low'], 
                      col="rep", size=3, aspect=1)

cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes

g = g.map_dataframe(facet_heatmap,
                    cbar_ax=cbar_ax,
                    vmin=0, vmax=1)  # <-- Specify the colorbar axes and limits

g.set_titles(col_template="{col_name}", fontweight='bold', fontsize=18)
g.fig.subplots_adjust(right=.9)  # <-- Add space so the colorbar doesn't overlap the plot

In [ ]:
def filter_by_abundance(data, column, high, low):
    '''Return only rows where genera have one value of abundance in range(low, high)'''
    species_to_keep = data[(data[column] <= high) & (data[column] >= low)]['Genus'].unique()
    print species_to_keep[0:5]# 
    return data[data['Genus'].isin(species_to_keep)]
    #return data[data['Genus'] in species_to_keep]

filter_by_abundance(data=data_reduced_genus, column='abundance', high=1, low=0.5).head()

In [ ]:
# Make a heat map.  Then we aren't confused by colors, and overwhelmed by the # of labels. 
d_plot = filter_by_abundance(data=data_reduced_genus, column='abundance', high=.8, low=0.1)

def facet_heatmap(data, color, **kws):
    data = data.pivot(index='Genus', columns='week', values='abundance')
    sns.heatmap(data, cmap="YlGnBu", **kws)  # <-- Pass kwargs to heatmap  cmap used to be 'Blue'

with sns.plotting_context(font_scale=7):
    g = sns.FacetGrid(d_plot[d_plot['oxy']=='Low'], 
                      col="rep", size=3, aspect=1)

cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes

g = g.map_dataframe(facet_heatmap,
                    cbar_ax=cbar_ax,
                    vmin=0, vmax=max(d_plot.abundance))  # <-- Specify the colorbar axes and limits

g.set_titles(col_template="{col_name}", fontweight='bold', fontsize=18)
g.fig.subplots_adjust(right=.9)  # <-- Add space so the colorbar doesn't overlap the plot

In [ ]:
def plot_heatmap(data, high, low, oxy, rep): 
    # get rid of oxygen levels and replicates if specified.
    if oxy is not 'all':
        print "keep only {} oxygen samples".format(oxy)
        data = data[data['oxy']==oxy]
    if rep is not 'all':
        print "keep only replicate levels:", rep
        data = data[data['rep'].isin(rep)]        
    data = filter_by_abundance(data=data, column='abundance', high=high, low=low)
    data['facet_replicate'] = 'replicate ' + data['rep'].astype(str)
    
    # make height of the plot a function of the number of rows (Genera):
    num_data_rows = len(data['Genus'].unique())
    plot_size = 2 +  num_data_rows/7
    plot_aspect = 2
    if num_data_rows > 6:
        plot_aspect = .85
    if num_data_rows > 9:
        plot_aspect = .65
    if num_data_rows > 9:
        plot_aspect = .6
    
    def facet_heatmap(data, color, **kws):
        data = data.pivot(index='Genus', columns='week', values='abundance')
        sns.heatmap(data, cmap="YlGnBu", **kws)  # <-- Pass kwargs to heatmap  cmap used to be 'Blue'
    
    with sns.plotting_context(font_scale=7):
        g = sns.FacetGrid(data, col='facet_replicate', size=plot_size, aspect=plot_aspect)
    
    cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
    
    g = g.map_dataframe(facet_heatmap,
                        cbar_ax=cbar_ax,
                        vmin=0, vmax=max(data.abundance))  # <-- Specify the colorbar axes and limits
    
    g.set_titles(col_template="{col_name}", fontweight='bold', fontsize=18)
    g.set_axis_labels('week')
    g.fig.subplots_adjust(right=.9)  # <-- Add space so the colorbar doesn't overlap the plot
    
    # add a supertitle. 
    plt.subplots_adjust(top=0.80)
    supertitle = str(low) + ' < abundance < ' + str(high) + ', {} oxygen'.format(oxy) 
    g.fig.suptitle(supertitle, size=14)
    
    # write a filename and save.
    plot_dir = './plots/'
    filename = oxy + "_oxygen--{0}_to_{1}_abundance".format(low,high)
    print 'filename:', filename
    g.savefig(plot_dir + filename + '.pdf')

# demo:
plot_heatmap(data=data_reduced_genus, high=0.6, low = 0.5, oxy='High', rep='all')

In [ ]:
plot_heatmap(data=data_reduced_genus, high=1, low = 0.1, oxy='High', rep='all')

In [ ]:
plot_heatmap(data=data_reduced_genus, high=.5, low = 0.02, oxy='Low', rep='all')

In [ ]:
# Loop through the ranges we want. 
oxy_conditions = ['Low', 'High']
range_pairs = [(0.01, 1), (0.05, 1), (0.1, 1), (0.5, 1)]
for o in oxy_conditions:
    for r in range_pairs:
        #print "oxygen:", o
        #print "range:", r
        #print "range (low):", r[0]
        plot_heatmap(data=data_reduced_genus, low =r[0], high=r[1], oxy=o, rep='all')

In [ ]:
# Loop through the ranges we want. 
oxy_conditions = ['Low', 'High']
max_abundance = 0.6
range_pairs = [(0.01, max_abundance), (0.05, max_abundance), (0.1, max_abundance), (0.5, max_abundance)]
for o in oxy_conditions:
    for r in range_pairs:
        #print "oxygen:", o
        #print "range:", r
        #print "range (low):", r[0]
        plot_heatmap(data=data_reduced_genus, low =r[0], high=r[1], oxy=o, rep='all')